In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import blitzbeaver as bb

In [ ]:
csv_path = "../../aptihramy/data/csv_cleaned"

start_year = 1805
end_year = 1810

dataframes = [
    pl.read_csv(f"{csv_path}/{year}.csv", infer_schema_length=10000)
    for year in range(start_year, end_year + 1)
]

In [ ]:
# load the graph from a .beaver file
path_graph = "../graph.beaver"

graph = bb.read_beaver(path_graph)

In [ ]:
record_schema = bb.RecordSchema(
    [
        bb.FieldSchema("nom_rue_norm", bb.ElementType.String),
        bb.FieldSchema("chef_prenom_norm", bb.ElementType.String),
        bb.FieldSchema("chef_nom_norm", bb.ElementType.String),
        bb.FieldSchema("chef_origine", bb.ElementType.String),
        bb.FieldSchema("epouse_nom", bb.ElementType.String),
        bb.FieldSchema("chef_vocation", bb.ElementType.String),
    ]
)

In [ ]:
distance_metric_config = bb.DistanceMetricConfig(
    metric="lv_opti",
    caching_threshold=4,
    lv_substring_weight=0.5,
)
normal_memory_config = bb.MemoryConfig(
    memory_strategy="median",
)
multi_memory_config = bb.MemoryConfig(
    memory_strategy="mw-median",
    multiword_threshold_match=0.6,
    multiword_distance_metric=distance_metric_config,
)

config = bb.TrackingConfig(
    num_threads=17,
    tracker=bb.TrackerConfig(
        interest_threshold=0.6,
        limit_no_match_streak=3,
        memories=(
            6 * [normal_memory_config]
            + [multi_memory_config]
        ),
        record_scorer=bb.RecordScorerConfig(
            record_scorer="weighted-average",
            weights=[
                0.15,
                0.25,
                0.25,
                0.1,
                0.1,
                0.1,
            ],
            min_weight_ratio=0.7,
        )
    ),
    distance_metric=bb.DistanceMetricConfig(
        metric="lv_opti",
        caching_threshold=4,
        lv_substring_weight=0.5,
    ),
    resolver=bb.ResolverConfig(
        resolving_strategy="best-match",
    ),
)

In [ ]:
# compute the graph
graph = bb.execute_tracking(config, record_schema, dataframes, "debug")

In [ ]:
def aggregate_histograms(histograms: list[int]) -> list[int]:
    """
    Aggregates a list of histograms into a single histogram.
    """
    max_len = max([len(h) for h in histograms])
    result = [0] * max_len
    for h in histograms:
        for i, v in enumerate(h):
            result[i] += v
    return result

def summary_graph(graph: bb.TrackingGraph):
    # compute the sum of the histograms for all frames
    histogram_records = aggregate_histograms([resolving.histogram_record_matchs for resolving in graph.diagnostics.resolvings])
    # do not show beyond 10 elements as the counts are very low
    histogram_records = histogram_records
    histogram_trackers = aggregate_histograms([resolving.histogram_tracker_matchs for resolving in graph.diagnostics.resolvings])
    histogram_trackers = histogram_trackers[:10]

    chain_metrics = bb.evaluate_tracking_chain_length(graph._raw)
    graph_metrics = bb.evaluate_tracking_graph_properties(graph._raw)

    records_match_ratios = graph_metrics.records_match_ratios[1:]
    trackers_match_ratios = graph_metrics.trackers_match_ratios[1:-1]
    avg_records_match = sum(records_match_ratios) / len(records_match_ratios)
    avg_trackers_match = sum(trackers_match_ratios) / len(trackers_match_ratios)

    per_divergence = sum(histogram_trackers[2:]) / sum(histogram_trackers[1:])
    per_conflict = sum(histogram_records[2:]) / sum(histogram_records[1:])

    # total number of trackers created
    print(f"Number of chains: {len(graph.trackers_ids)}")
    # average percentage of records that have been match with an existing tracker
    print(f"Percentage of matching records: {avg_records_match*100:.2f}%")
    # average percentage of trackers that have match with a record of the current frame
    print(f"Percentage of matching trackers: {avg_trackers_match*100:.2f}%")
    # number of times a tracker matched with more that one record
    # divided by the number of times a tracker matched with a record
    print(f"Percentage of divergences: {per_divergence*100:.2f}%")
    # number of times a record matched with multiple trackers
    # divided by the number of times a record matched with a tracker
    print(f"Percentage of conflicts: {per_conflict*100:.2f}%")

    plt.bar(range(1, len(chain_metrics.histogram)), chain_metrics.histogram[1:])
    plt.title("Histogram of tracking chain lengths")
    plt.show()

    plt.bar(range(len(histogram_records)), histogram_records)
    plt.title("Histogram of # matchs per record")
    plt.show()

    plt.bar(range(len(histogram_trackers)), histogram_trackers)
    plt.title("Histogram of # matchs per tracker")
    plt.show()

    

In [ ]:
summary_graph(graph)

In [ ]:
path_graph = "../graph.beaver"

bb.save_beaver(path_graph, graph)

In [ ]:
chain_id = graph.trackers_ids[1293]

chain = graph.materialize_tracking_chain(chain_id, dataframes, record_schema)
chain.as_dataframe()

In [ ]:
words = [
    "magimelien",
    "mazimilien",
    "mazirelien",
    "marinelien",
]
bb.compute_median_word(words)
